<a href="https://colab.research.google.com/github/gidly24/gau/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import random
from math import fabs

Nix,Niy=5,7
Ni=Nix*Niy
Nout=6
N1=int((Ni+Nout)/2)
Ntest=7
Alpha=2
Beta=2   # 1
Nju=0.2  # 0.1
Mju=0.1
noiseLevel=0.15 #0.25
noiseTrain=0.5

inp1=np.zeros(Ni)
w1=np.ones((N1,Ni+1))
layer1=np.zeros(N1)
delta1=np.zeros((N1,Ni+1))
w2=np.ones((Nout,N1+1))
layer2=np.zeros(Nout)
delta2=np.zeros((Nout,N1+1))

weights=[w1,w2]

#print(inp1)
#print(w1)

def initWeights(w):
  for i in range(len(w)):
    w[i]=random.random()-0.5

def vvod(source,dest):
  for i in range(len(source)):
    dest[i]=source[i]

def noise(arr,n):
  for i in range(len(arr)):
    a=random.random()
    if a<n:
      if arr[i]==1:
        arr[i]=0
      else:
        arr[i]=1

def showImage(arr,X,Y):
  a1=np.array(arr).reshape(Y,X)
  for y in range(Y):
    for x in range(X):
      if a1[y][x]==1:
        print('#',end='')
      else:
        print('.',end='')
    print()

def work(inp,wa,out,alpha):
  for o in range(len(out)):
    sum=0.0
    for i in range(len(inp)):
      sum+=inp[i]*wa[o][i]
    sum+=wa[o][len(inp)]
    out[o]=1/(1+np.exp(-sum*alpha))

'''
#def train(ideal,inp,wa,out,delta,beta,nju):
def train(ideal,inp,wa,out,beta,nju):
  nout=len(out)
  ninp=len(inp)
  err=np.zeros(nout)
  for k in range(nout):  # Vychislenie oshibki vyhodnogo sloja
    err[k]=(ideal[k]-out[k])*beta*out[k]*(1-out[k])
    wa[1][k][ninp]+=nju*err[k]
  for j in range(ninp):
    for k in range(nout):
#      delta[k][j]=nju*err[k]*inp[j]
      delta=nju*err[k]*inp[j]
      wa[1][k][j]+=delta#[k][j]
#'''

def train(ideal,wa,layers,deltas,arch,beta,Nju,Mju):
#  Work(axon0); // Prohozhdenie signala cherez set'
  N0=arch[0]
  N1=arch[1]
  N2=arch[2]
  out=layers[2]
  axon0=layers[0]
  axon1=layers[1]
  sinaps2=wa[1]
  sinaps1=wa[0]
  err2=np.zeros(N2)
  err1=np.zeros(N1)

  for k in range(N2): # Vychislenie oshibki vyhodnogo sloja
    err2[k]=(ideal[k]-out[k])*beta*out[k]*(1-out[k]);
    wa[1][k][N1]+=Nju*err2[k];  # smesh
  for j in range(N1): # Vychislenie oshibki skrytogo sloja
    s=0.;
    for k in range(N2):
      s+=err2[k]*sinaps2[k][j];
    err1[j]=s*beta*axon1[j]*(1-axon1[j]);
  for i in range(N0):
    for j in range(N1):
      deltas[0][j][i]=Nju*(Mju*deltas[0][j][i]+(1-Mju)*err1[j]*axon0[i]);
      sinaps1[j][i]+=deltas[0][j][i];

  for j in range(N1):
    for k in range(N2):
      deltas[1][k][j]=Nju*(Mju*deltas[1][k][j]+(1-Mju)*err2[k]*axon1[j]);
      sinaps2[k][j]+=deltas[1][k][j];
    wa[0][j][N0]+=Nju*err1[j];  # smesh

def isConverged(ideal,out,prec):
    c=0
    for k in range(len(out)):
      if fabs(ideal[k]-out[k])<=prec:
        c+=1
    return c #'''

signal1=[[
   1,0,0,0,1,
   1,0,0,0,1,
   1,1,0,0,1,
   1,0,1,0,1,
   1,0,0,1,1,
   1,0,0,0,1,
   1,0,0,0,1],

  [1,1,1,0,0,
   1,0,0,1,0,
   1,0,0,1,0,
   1,1,1,0,0,
   1,0,0,0,0,
   1,0,0,0,0,
   1,0,0,0,0],

  [0,0,1,0,0,
   0,1,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,1,1,1,0],

  [0,1,1,0,0,
   1,0,0,1,0,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,0,0,0,
   1,0,0,0,0,
   1,1,1,1,0],

  [1,1,1,1,0,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,1,0,0,
   0,0,0,1,0,
   1,0,0,1,0,
   0,1,1,0,0],

  [1,1,1,1,1,
   0,0,0,0,1,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,0,0,0,
   1,0,0,0,0,
   1,0,0,0,0]
  ]

signal2=[[
   1,0,0,0,1,
   1,0,0,0,1,
   1,1,0,0,1,
   1,0,1,0,1,
   1,0,0,1,1,
   1,0,0,0,1,
   1,0,0,0,1],

  [1,1,1,0,0,
   1,0,0,1,0,
   1,0,0,1,0,
   1,1,1,0,0,
   1,0,0,0,0,
   1,0,0,0,0,
   1,0,0,0,0],

  [0,0,1,0,0,
   0,1,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,1,1,1,0],

  [0,1,1,0,0,
   1,0,0,1,0,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,0,0,0,
   1,0,0,0,0,
   1,1,1,1,0],

  [1,1,1,1,0,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,1,0,0,
   0,0,0,1,0,
   1,0,0,1,0,
   0,1,1,0,0],

  [1,1,1,1,1,
   0,0,0,0,1,
   0,0,0,1,0,
   0,0,1,0,0,
   0,1,0,0,0,
   1,0,0,0,0,
   1,0,0,0,0],

  [0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0,
   0,0,1,0,0],
  ]

netOut=[
  [1,0,0,0,0,0],
  [0,1,0,0,0,0],
  [0,0,1,0,0,0],
  [0,0,0,1,0,0],
  [0,0,0,0,1,0],
  [0,0,0,0,0,1]]

metki=['N','P','1','2','3','7','1']

'''
print('Тест обучающего образа:')
showImage(signal2[0],Nix,Niy); print()
input('Нажмите <Enter> для продолжения.')

print('Тест входного образа:')
vvod(signal1[0],inp1)
showImage(inp1,Nix,Niy); print()
print('Он же с шумом:')
noise(inp1,noiseLevel)
showImage(inp1,Nix,Niy); print()
input('Нажмите <Enter> для продолжения.')

print('Контроль входного сигнала в виде одномерного массива:',inp1)
#print('Image_1\n',inp1)
input('Нажмите <Enter> для продолжения.')
#'''

work(inp1,w1,layer1,Alpha)
work(layer1,w2,layer2,Alpha)
print('Проверка состояния одного выхода сети (сеть ещё не обучена):',layer2[0])
#print('Out\n',layer1[0])
input('Нажмите <Enter> для продолжения.')

for i in range(N1):
  initWeights(w1[i])
for i in range(Nout):
  initWeights(w2[i])

print('Все весовые коэффициенты 1-слойной сети, w1:\n',w1,'w2:\n',w2)
input('Нажмите <Enter> для продолжения.')

work(inp1,w1,layer1,Alpha)
work(layer1,w2,layer2,Alpha)
print('Out\n',layer1[0])
input('Нажмите <Enter> для продолжения.')

#Corr=0
Corr=np.zeros(Nout)
Corr1=0
eps=0.02
epoch=0
input('Процесс обучения сети. Нажмите <Enter> для продолжения.')
while Corr1<Nout:
  a=random.random()
#  print(a)
  for i in range(len(netOut)):
    vvod(signal1[i],inp1)
    if a<noiseTrain:
      noise(inp1,noiseLevel)

    work(inp1,w1,layer1,Alpha)
    work(layer1,w2,layer2,Alpha)
#    train(netOut[i],inp1,weights,layer1,Beta,Nju)
    train(netOut[i],weights,[inp1,layer1,layer2],[delta1,delta2],[Ni,N1,Nout],Beta,Nju,Mju)
    Corr[i]=isConverged(netOut[i],layer2,eps)
    Corr1=0
    for c in range(Nout):
      if Corr[c]==Nout:
        Corr1+=1
  epoch+=1
  print('Epoch=',epoch,'No errors=',Corr)
#  print(delta1)

print('Число циклов обучения=',epoch)
print('No errors=',Corr)
print('Test signals:')
for i in range(Ntest):
  print('Signal-',i,':',metki[i],':')
  showImage(signal2[i],Nix,Niy)
  vvod(signal2[i],inp1)
  work(inp1,w1,layer1,Alpha)
  work(layer1,w2,layer2,Alpha)
  for j in range(Nout):
    print('Out-',j,'=',layer2[j])
  print()

input('Continue.')

print('Test signals with noise:')
for i in range(Ntest):
  print('Signal-',i,':',metki[i],':')
  vvod(signal2[i],inp1)
  noise(inp1,0.125)
  showImage(inp1,Nix,Niy)
  work(inp1,w1,layer1,Alpha)
  work(layer1,w2,layer2,Alpha)
  for j in range(Nout):
    print('Out-',j,'=',layer2[j])
  print()

input('Press <Enter>.')


Проверка состояния одного выхода сети (сеть ещё не обучена): 1.0
Нажмите <Enter> для продолжения.
Все весовые коэффициенты 1-слойной сети, w1:
 [[-0.25650682  0.46057252 -0.32235457 -0.46573285  0.4919412   0.38598956
   0.32890026 -0.13266942  0.09650384 -0.4094322  -0.26597985  0.09976368
  -0.1524951  -0.38397761 -0.4584668  -0.27881363  0.43235226 -0.43500333
  -0.30161086  0.29675416  0.32885458 -0.15219046 -0.49936359 -0.09362805
  -0.37763892 -0.30554011 -0.20389062 -0.37097898  0.45705879  0.05138172
   0.41508095 -0.22218782  0.08795812  0.07607153 -0.41470235 -0.08465422]
 [ 0.32266214 -0.25756043 -0.1164437  -0.49015566 -0.11854543  0.22020929
  -0.21160703  0.03882893  0.43690932 -0.18609245 -0.39869342  0.43572582
  -0.19850038  0.36309868  0.18550515 -0.34436392  0.15601893  0.02463759
   0.02943891 -0.37374289 -0.27005143 -0.36864634 -0.42083123  0.47764068
   0.28864435 -0.35481631  0.49683072  0.44905603 -0.27104896 -0.42220205
   0.01850271  0.45997065  0.20180228  0.

''